In [1]:
import warnings
import tensorflow as tf
import glob
from tqdm import tqdm
import midi_manipulation
import numpy as np
from tensorflow.python.ops import control_flow_ops
from distutils.version import LooseVersion
from utils import *



input_songs, target_songs = get_song_matrixes('./blues', 30, 300)

input_songs = np.array(input_songs)
target_songs = np.array(target_songs)


tokens = get_tokens(input_songs)
num_encoder_tokens = np.array(tokens).shape[0]
num_decoder_tokens = np.array(tokens).shape[0]


print('[*] Embedding Songs')
embeded_input_songs = get_embeded_songs(input_songs, tokens, num_encoder_tokens)
embeded_target_songs = get_embeded_songs(input_songs, tokens, num_encoder_tokens)

print(np.array(embeded_input_songs[0]).shape)


print(np.array(embed_song_to_song(embeded_input_songs[0], tokens)).shape)

# Finding the longest song in the dataset
max_encoder_seq_length = max([len(song) for song in embeded_input_songs])
max_decoder_seq_length = max([len(song) for song in embeded_target_songs])


print('Number of samples:', len(input_songs))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)


# Get input data in shape (num_sample, max_seq_length, num_tokens)
encoder_input_data, decoder_input_data, decoder_target_data = get_input_data(
                                                                        embeded_input_songs, 
                                                                        embeded_target_songs,
                                                                        max_encoder_seq_length, 
                                                                        num_encoder_tokens, 
                                                                        max_decoder_seq_length, 
                                                                        num_decoder_tokens)

/Users/dam/anaconda3/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
  0%|          | 0/180 [00:00<?, ?it/s]

[*] Converting songs to matrix


 16%|█▌        | 29/180 [00:21<01:52,  1.35it/s]

[*] Converted 30 songs to matrix



(300, 156)
Processing song, 0
Processing song, 1
Processing song, 2
Processing song, 3
Processing song, 4
Processing song, 5
Processing song, 6
Processing song, 7
Processing song, 8
Processing song, 9
Processing song, 10
Processing song, 11
Processing song, 12
Processing song, 13
Processing song, 14
Processing song, 15
Processing song, 16
Processing song, 17
Processing song, 18
Processing song, 19
Processing song, 20
Processing song, 21
Processing song, 22
Processing song, 23
Processing song, 24
Processing song, 25
Processing song, 26
Processing song, 27
Processing song, 28
Processing song, 29
Processing song, 30
Processing song, 31
Processing song, 32
Processing song, 33
Processing song, 34
Processing song, 35
Processing song, 36
Processing song, 37
Processing song, 38
Processing song, 39
Processing song, 40
Processing song, 41
Processing song, 42
Processing song, 43
Processing song, 44
Processing song, 45
Processing song, 46
Processing song, 47
Pro

In [2]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense

# Hyperparameters
batch_size  =   16      # Batch size for training.
epochs      =   15     # Number of epochs to train for.
latent_dim  =   256     # Latent dimensionality of the encoding space.
#num_samples =   10000   # Number of samples to train on.


# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]


# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)



# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
print()
print(model.summary())
print()

Using TensorFlow backend.



__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 6866)   0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 6866)   0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 256), (None, 7293952     input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 256),  7293952     input_2[0][0]                    
                                                                 lstm_1[0][1]                     
         

In [3]:
# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

print('[*] Starting Training')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# model.load_weights('s2s.h5')
print('[*] Ready to be used \n\n')
# Save model
model.save('s2s.h5')

[*] Starting Training
Train on 43 samples, validate on 11 samples
Epoch 1/15
43/43 [==============================] - 140s 3s/step - loss: 8.8549 - acc: 0.0644 - val_loss: 9.1782 - val_acc: 0.0085
Epoch 2/15
43/43 [==============================] - 125s 3s/step - loss: 8.2936 - acc: 0.1088 - val_loss: 9.9277 - val_acc: 0.0085
Epoch 3/15
32/43 [=====================>........] - ETA: 26s - loss: 7.6032 - acc: 0.1226 

KeyboardInterrupt: 

In [4]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)


# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [6]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, 0] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    song_matrix = np.zeros(
                        (max_decoder_seq_length, 
                        num_decoder_tokens),
                        dtype='float32')
    i = 0
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        idx = np.argmax(output_tokens[-1,-1,:])
        #print(output_tokens[-1,-1,:])
        song_matrix[i, idx] = 1
        target_seq[0, 0, idx] = 1.
        #print(np.array(h).shape)
        # Exit condition: either hit max length
        # or find stop character.
        if (i+2 > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        
        

        if i%250 == 0:
            print('[iter:{}] [max_decoder_seq_length: {}]'.format(i, max_decoder_seq_length))
        i+=1

        # Update states
        states_value = [h, c]

    return song_matrix


seq_length = max_encoder_seq_length

song = np.array(midi_manipulation.midiToNoteStateMatrix('./blues/BB_King_-_Sweet_Sixteen._mid_'))

encoder_input_data = []
if np.array(song).shape[0] > 50:   
    length = np.array(song).shape[0]
    for j in range(length // seq_length):
        encoder_input_data.append(song[seq_length*j:seq_length*(j+1)])
encoder_input_data = get_embeded_songs(encoder_input_data, tokens, num_encoder_tokens)                                                       


# Take one sequence (part of the training test)
# for trying out decoding.
print('[*] Encoding-Decoding')
input_seq = encoder_input_data[0:-1]
decoded_song = decode_sequence(input_seq)

decoded_song = embed_song_to_song(decoded_song, tokens)

# Converting Song to midi from matrix
print('[*] Converting and saving song')
midi_manipulation.noteStateMatrixToMidi(decoded_song)

Embedding song 0
Embedding song 1
Embedding song 2
Embedding song 3
[*] Encoding-Decoding
[iter:0] [max_decoder_seq_length: 300]
[iter:250] [max_decoder_seq_length: 300]
[*] Converting and saving song
